Code idea from

https://www.alpharithms.com/relative-strength-index-rsi-in-python-470209/

https://github.com/alpharithms/data/blob/main/btc-usd-20210101-20210701.csv

https://github.com/alpharithms/data/blob/main/wilder-rsi-data.csv

This code is for RSI in 3 different ways:
1. traditional maths
2. panda
3. panda_ta

visualization and validation is not in function for this code

In [1169]:
import csv

# Load the data via Python's CSV module
with open('wilder-rsi-data.csv', 'r')as file:
    reader = csv.reader(file)
    header = next(reader)  # skip the header
    data = [row[1] for row in list(reader)]
# View the Data
print (data)
# Our data, as a native Python List
#['54.8', '56.8', '57.85', '59.85', '60.57', '61.1', '62.17', '60.6', '62.35', '62.15', '62.35', '61.45', '62.8', '61.37', '62.5', '62.57', '60.8', '59.37', '60.35', '62.35', '62.17', '62.55', '64.55', '64.37', '65.3', '64.42', '62.9', '61.6', '62.05', '60.05', '59.7', '60.9', '60.25', '58.27', '58.7', '57.72', '58.1', '58.2']

# Define our Lookback period (our sliding window)
window_length = 14
# Initialize containers for avg. gains and losses
gains = []
losses = []
# Create a container for current lookback prices
window = []
# Keeps track of previous average values
prev_avg_gain = None
prev_avg_loss = None
# Create a container for our final output (as a csv)
output = [['date', 'close', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rsi']]

#wilder_data = data
#wilder_data = [int(numeric_string) for numeric_string in data]
wilder_data = []
for i in range(len(data)):
    t = float(data[i])
    wilder_data.append(t)

# Loop through an enumerated set of our data
# to keep track of which period we are currently
# making calculations for.
for i, price in enumerate(wilder_data):
    # keep track of the price for the first period
    # but don't calculate a difference value.
    if i == 0:
        window.append(price)
        output.append([i+1, price, 0, 0, 0, 0, 0])
        continue
    # After the first period, calculate the difference
    # between price and previous price as a rounded value
    difference = round(wilder_data[i] - wilder_data[i - 1], 2)
    print(difference, end=' ')

    # Record positive differences as gains
    if difference > 0:
        gain = difference
        loss = 0
    # Record negative differences as losses
    elif difference < 0:
        gain = 0
        loss = abs(difference)
    # Record no movements as neutral
    else:
        gain = 0
        loss = 0
    # Save gains/losses
    gains.append(gain)
    losses.append(loss)
    # Continue to iterate until enough
    # # gains/losses data is available to 
    # # calculate the initial RS value
    if i < window_length:
        window.append(price)
        output.append([i+1, price, gain, loss, 0, 0, 0])
        continue
    # Calculate SMA for first gain
    if i == window_length:
        avg_gain = sum(gains) / len(gains)
        avg_loss = sum(losses) / len(losses)
    # Use WSM after initial window-length period
    else:
        avg_gain = (prev_avg_gain * (window_length - 1) + gain) / window_length
        avg_loss = (prev_avg_loss * (window_length - 1) + loss) / window_length
    # Keep in memory
    prev_avg_gain = avg_gain
    prev_avg_loss = avg_loss
    # Round for later comparison (optional)
    avg_gain = round(avg_gain, 2)
    avg_loss = round(avg_loss, 2)
    prev_avg_gain = round(prev_avg_gain, 2)
    prev_avg_loss = round(prev_avg_loss, 2)

rs = round(avg_gain / avg_loss, 2)
rsi = round(100 - (100 / (1 + rs)), 2)

# Remove oldest values
window.append(price)
window.pop(0)
gains.pop(0)
losses.pop(0)
# Save Data
output.append([i+1, price, gain, loss, avg_gain, avg_loss, rsi])

output

['54.8', '56.8', '57.85', '59.85', '60.57', '61.1', '62.17', '60.6', '62.35', '62.15', '62.35', '61.45', '62.8', '61.37', '62.5', '62.57', '60.8', '59.37', '60.35', '62.35', '62.17', '62.55', '64.55', '64.37', '65.3', '64.42', '62.9', '61.6', '62.05', '60.05', '59.7', '60.9', '60.25', '58.27', '58.7', '57.72', '58.1', '58.2']
2.0 1.05 2.0 0.72 0.53 1.07 -1.57 1.75 -0.2 0.2 -0.9 1.35 -1.43 1.13 0.07 -1.77 -1.43 0.98 2.0 -0.18 0.38 2.0 -0.18 0.93 -0.88 -1.52 -1.3 0.45 -2.0 -0.35 1.2 -0.65 -1.98 0.43 -0.98 0.38 0.1 

[['date', 'close', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rsi'],
 [1, 54.8, 0, 0, 0, 0, 0],
 [2, 56.8, 2.0, 0, 0, 0, 0],
 [3, 57.85, 1.05, 0, 0, 0, 0],
 [4, 59.85, 2.0, 0, 0, 0, 0],
 [5, 60.57, 0.72, 0, 0, 0, 0],
 [6, 61.1, 0.53, 0, 0, 0, 0],
 [7, 62.17, 1.07, 0, 0, 0, 0],
 [8, 60.6, 0, 1.57, 0, 0, 0],
 [9, 62.35, 1.75, 0, 0, 0, 0],
 [10, 62.15, 0, 0.2, 0, 0, 0],
 [11, 62.35, 0.2, 0, 0, 0, 0],
 [12, 61.45, 0, 0.9, 0, 0, 0],
 [13, 62.8, 1.35, 0, 0, 0, 0],
 [14, 61.37, 0, 1.43, 0, 0, 0],
 [38, 58.2, 0.1, 0, 0.42, 0.53, 44.13]]

Starting from below, panda is used to compute RSI. 

First, load the data

In [1170]:
import pandas as pd

import numpy as np
import requests
import matplotlib.pyplot as plt
from math import floor
from termcolor import colored as cl

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20,10)

from binance.client import Client
import datetime as dt
import json
# Load our sample CSV data as DataFrame
df = pd.read_csv('./wilder-rsi-data.csv', header=0).set_index(['period'])
# View the result
#print(df.head())

# Calculate Price Differences
df['diff'] = df.diff(1)
# View Results
#print(df)
# Calculate Avg. Gains/Losses
df['gain'] = df['diff'].clip(lower=0).round(2)
df['loss'] = df['diff'].clip(upper=0).abs().round(2)
# View Result
#print(df)
window_length = 14

# Get initial Averages
df['avg_gain'] = df['gain'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]
df['avg_loss'] = df['loss'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]
# View Result
#print(df)
# View first SMA value
#print(df.iloc[window_length-1: window_length+2])

# Get WMS averages
# Average Gains
for i, row in enumerate(df['avg_gain'].iloc[window_length+1:]):
    df['avg_gain'].iloc[i + window_length + 1] =\
        (df['avg_gain'].iloc[i + window_length] *
         (window_length - 1) +
         df['gain'].iloc[i + window_length + 1])\
        / window_length
# Average Losses
for i, row in enumerate(df['avg_loss'].iloc[window_length+1:]):
    df['avg_loss'].iloc[i + window_length + 1] =\
        (df['avg_loss'].iloc[i + window_length] *
         (window_length - 1) +
         df['loss'].iloc[i + window_length + 1])\
        / window_length
# View initial results
#print(df[window_length-1:window_length+5])
# Calculate RS Values
df['rs'] = df['avg_gain'] / df['avg_loss']

# Calculate RSI
df['rsi'] = 100 - (100 / (1.0 + df['rs']))
# View Result
print(df)

        price  diff  gain  loss  avg_gain  avg_loss        rs        rsi
period                                                                  
1       54.80   NaN   NaN   NaN       NaN       NaN       NaN        NaN
2       56.80  2.00  2.00  0.00       NaN       NaN       NaN        NaN
3       57.85  1.05  1.05  0.00       NaN       NaN       NaN        NaN
4       59.85  2.00  2.00  0.00       NaN       NaN       NaN        NaN
5       60.57  0.72  0.72  0.00       NaN       NaN       NaN        NaN
6       61.10  0.53  0.53  0.00       NaN       NaN       NaN        NaN
7       62.17  1.07  1.07  0.00       NaN       NaN       NaN        NaN
8       60.60 -1.57  0.00  1.57       NaN       NaN       NaN        NaN
9       62.35  1.75  1.75  0.00       NaN       NaN       NaN        NaN
10      62.15 -0.20  0.00  0.20       NaN       NaN       NaN        NaN
11      62.35  0.20  0.20  0.00       NaN       NaN       NaN        NaN
12      61.45 -0.90  0.00  0.90       NaN       NaN

/var/folders/vm/cgbfh7n92sn3jkyx9_t6h3qh0000gn/T/ipykernel_2576/3557734325.py:32: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

/var/folders/vm/cgbfh7n92sn3jkyx9_t6h3qh0000gn/T/ipykernel_2576/3557734325.py:33: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.



In [1171]:

# Validation list
WILDER_RSI_VALUES = [
    74.36, 74.55, 65.75, 59.68, 61.98, 66.44, 65.75, 67.0, 71.43,
    70.5, 72.14, 67.95, 60.78, 55.56, 56.71, 49.49, 48.19, 52.38,
    50.0, 43.5, 45.36, 42.53, 44.14, 44.75
]
# Load in the validation data + create NaN values preceding our first RSI calculations
v = pd.DataFrame(pd.concat([pd.Series(["NaN"] * (window_length)), pd.Series(WILDER_RSI_VALUES)])).reset_index(level=0).drop(['index'], axis=1)
v.index = list(range(1, len(v) + 1))  # reindex starting with 0
# Calculate differences
df['diff_rsi'] = ((df['rsi'] - v.values).abs())
df['diff_pct'] = ((df['rsi'] - v.values) / v.values * 100).abs()
# Round off for easy comparison
df['diff_rsi'] = df['diff_rsi'].apply(lambda x: round(x, 2))
df['diff_pct'] = df['diff_pct'].apply(lambda x: round(x, 2))
# View Results
#print(df)

Below is for Calculating the RSI with Pandas_TA. It is self-sufficient. 


In [1172]:
import pandas as pd
import pandas_ta as ta
# Load the data

length = 14

url = 'https://api.binance.com/api/v3/klines'
symbol = 'ADAUSDT'
interval = '1d'
start = str(int(dt.datetime(2022,3,1).timestamp()*1000))
end = str(int(dt.datetime(2022,11,1).timestamp()*1000))
par = {'symbol': symbol, 'interval': interval, 'startTime': start, 'endTime': end}
data = pd.DataFrame(json.loads(requests.get(url, params= par).text))
#format columns name
data.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.datetime]
data=data.astype(float)
df = data

#df = pd.read_csv('wilder-rsi-data.csv', header=0).set_index(['period'])
# Calculate the RSI via pandas_ta
df.ta.rsi(close='price', length=length, append=True)
# View the result
print(df)

# Calculate with signal_indicators
#df.ta.rsi(close='price', length=14, append=True, signal_indicators=True)
# define custom overbought/oversold thresholds
df.ta.rsi(close='close', length=14, append=True, signal_indicators=True, xa=60, xb=40)

[X] Ooops!!! It's True, the series 'price' was not found in datetime, open, high, low, close, volume, close_time, qav, num_trades, taker_base_vol, taker_quote_vol, ignore
                         datetime    open    high     low   close  \
2022-03-01 08:00:00  1.646093e+12  0.9620  1.0110  0.9390  0.9630   
2022-03-02 08:00:00  1.646179e+12  0.9640  0.9770  0.9270  0.9390   
2022-03-03 08:00:00  1.646266e+12  0.9380  0.9460  0.8830  0.9030   
2022-03-04 08:00:00  1.646352e+12  0.9020  0.9030  0.8260  0.8440   
2022-03-05 08:00:00  1.646438e+12  0.8440  0.8830  0.8180  0.8640   
...                           ...     ...     ...     ...     ...   
2022-10-27 08:00:00  1.666829e+12  0.4030  0.4194  0.3861  0.3884   
2022-10-28 08:00:00  1.666915e+12  0.3884  0.4115  0.3777  0.4044   
2022-10-29 08:00:00  1.667002e+12  0.4044  0.4400  0.4008  0.4200   
2022-10-30 08:00:00  1.667088e+12  0.4201  0.4216  0.3981  0.4056   
2022-10-31 08:00:00  1.667174e+12  0.4057  0.4163  0.3975  0.4055   



,RSI_14,RSI_14_A_60,RSI_14_B_40
2022-03-01 08:00:00,NaN,0,0
2022-03-02 08:00:00,NaN,0,0
2022-03-03 08:00:00,NaN,0,0
2022-03-04 08:00:00,NaN,0,0
2022-03-05 08:00:00,NaN,0,0
...,...,...,...
2022-10-27 08:00:00,49.166236,0,0
2022-10-28 08:00:00,54.691850,0,0
2022-10-29 08:00:00,59.333327,0,0
2022-10-30 08:00:00,53.849495,0,0


In [1173]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create Figure
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_width=[0.25, 0.75])
# Inspect Result
print(fig)

data = df

print(data)

fig.add_trace(go.Candlestick(
    x=data.index,
    open=data['open'],
    high=data['high'],
    low=data['low'],
    close=data['close'],
    increasing_line_color='#ff9900',
    decreasing_line_color='black',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=data['RSI_14'],
    line=dict(color='#ff9900', width=2),
    showlegend=False,
), row=2, col=1
)

# Add upper/lower bounds
fig.update_yaxes(range=[-10, 110], row=2, col=1)
fig.add_hline(y=0, col=1, row=2, line_color="#666", line_width=2)
fig.add_hline(y=100, col=1, row=2, line_color="#666", line_width=2)
# Add overbought/oversold
fig.add_hline(y=30, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')
fig.add_hline(y=70, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')
# Customize font, colors, hide range slider
layout = go.Layout(
    plot_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    )
)
# update and display
fig.update_layout(layout)
fig.show()


Figure({
    'data': [],
    'layout': {'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'matches': 'x2', 'showticklabels': False},
               'xaxis2': {'anchor': 'y2', 'domain': [0.0, 1.0]},
               'yaxis': {'anchor': 'x', 'domain': [0.3625, 1.0]},
               'yaxis2': {'anchor': 'x2', 'domain': [0.0, 0.2125]}}
})
                         datetime    open    high     low   close  \
2022-03-01 08:00:00  1.646093e+12  0.9620  1.0110  0.9390  0.9630   
2022-03-02 08:00:00  1.646179e+12  0.9640  0.9770  0.9270  0.9390   
2022-03-03 08:00:00  1.646266e+12  0.9380  0.9460  0.8830  0.9030   
2022-03-04 08:00:00  1.646352e+12  0.9020  0.9030  0.8260  0.8440   
2022-03-05 08:00:00  1.646438e+12  0.8440  0.8830  0.8180  0.8640   
...                           ...     ...     ...     ...     ...   
2022-10-27 08:00:00  1.666829e+12  0.4030  0.4194  0.3861  0.3884   
2022-10-28 08:00:00  1.666915e+12  0.3884  0.4115  0.3777  0.4044   
2022-10-29 08